# Titanic - EasyEnsembleClassifier

https://machinelearningmastery.com/bagging-and-random-forest-for-imbalanced-classification/

Importing all necessary packages

In [1]:
# Version Check
from platform import python_version
python_version()

'3.7.6'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

### Task 1: Load Data from 'titanic_clean.csv'

In [3]:
#write code here
data = pd.read_csv('datasets/titanic_clean.csv')
df = data.copy()
df.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Embarked,Title,GrpSize,FareCat,AgeCat
0,1,0,3,male,1,0,S,Mr,couple,0-10,16-32
1,2,1,1,female,1,0,C,Mrs,couple,70-100,32-48
2,3,1,3,female,0,0,S,Miss,solo,0-10,16-32
3,4,1,1,female,1,0,S,Mrs,couple,40-70,32-48
4,138,0,1,male,1,0,S,Mr,couple,40-70,32-48


In [4]:
print(df.shape)

(891, 11)


### One Hot encoding for categorical varaibles

In [5]:
df_OneHot=pd.get_dummies(df,columns=['Pclass','Sex','Embarked','Title','GrpSize','FareCat','AgeCat'])
df_OneHot.head()

,PassengerId,Survived,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,...,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,FareCat_Oct-25,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
0,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,2,1,1,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,1,0,0
2,3,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,1,1,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,138,0,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0


In [6]:
df=df_OneHot.copy()

### Task 2: Create Independent and Dependent Variables

In [7]:
#write code here
X = df.drop(columns= ['Survived','PassengerId'], axis =1)
Y = df['Survived']

### Task 3: Train Test n Split the data

In [8]:
# Import the library
from sklearn.model_selection import train_test_split

In [40]:
#Write the code here
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.2, random_state=25, shuffle = True)
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

(712, 30) (712,)
(179, 30) (179,)


In [41]:
xtrain.head()

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,FareCat_Oct-25,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
410,1,2,1,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
638,1,0,1,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,1,0
395,0,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,1,0,0
221,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,1,0,1,0,0,0
800,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [42]:
ytrain.head()

410    0
638    0
395    0
221    0
800    0
Name: Survived, dtype: int64

# EasyEnsembleClassifier

### Creating Model & Training

In [43]:
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import RandomizedSearchCV

In [44]:
#Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=13, stop=14, num=2)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 50, num=5)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(1, 20, num=5)]

criterion = ["gini", "entropy"]

#max_features = ["auto", "sqrt", "log2"]
max_features = ["auto"]

In [45]:
random_grid1 = {"n_estimators": n_estimators,
                #"criterion": criterion,
                #"max_depth": max_depth,
                #"max_features": max_features,
                #"min_samples_split": min_samples_split,
               #"class_weight":['balanced_subsample','balanced']
               }

In [46]:
clf_rf = EasyEnsembleClassifier(n_estimators=5000,random_state=100, warm_start=True)
rf_random = RandomizedSearchCV(
    estimator=clf_rf, param_distributions=random_grid1, n_iter=500, cv=5, verbose=1, random_state=100,n_jobs=4)
# Fit the random search model
rf_random.fit(xtrain, ytrain)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=500. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    8.2s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=EasyEnsembleClassifier(base_estimator=None,
                                                    n_estimators=5000,
                                                    n_jobs=None,
                                                    random_state=100,
                                                    replacement=False,
                                                    sampling_strategy='auto',
                                                    verbose=0,
                                                    warm_start=True),
                   iid='deprecated', n_iter=500, n_jobs=4,
                   param_distributions={'n_estimators': [13, 14]},
                   pre_dispatch='2*n_jobs', random_state=100, refit=True,
                   return_train_score=False, scoring=None, verbose=1)

In [47]:
clf_rf_rand = rf_random.best_estimator_
clf_rf_rand.fit(xtrain, ytrain)

D:\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:352: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


EasyEnsembleClassifier(base_estimator=None, n_estimators=13, n_jobs=None,
                       random_state=100, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=True)

In [48]:
pred_rf_rand = clf_rf_rand.predict(xtest)


First value in Numpy array is probability of **0** and second is probability of **1**

Only predicting and extracting probability values of **1**

In [49]:
prb_rf_rand = clf_rf_rand.predict_proba(xtest)[:, 1] #probabilities of 1 (survival)

In [50]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

In [51]:
rf_random_recall= recall_score(ytest,pred_rf_rand)
rf_random_acc= accuracy_score(ytest,pred_rf_rand)
rf_random_pre= precision_score(ytest,pred_rf_rand)
rf_random_f1_score= f1_score(ytest,pred_rf_rand)

print("Random Forest Hyperparameters Tuning Using Randomized Search")
print("Precision: {0:.3f}%".format(rf_random_pre*100))
print("Accuracy:  {0:.3f}%".format(rf_random_acc*100))
print("Recall:    {0:.3f}%".format(rf_random_recall*100))
print("F1-Score:  {0:.3f}%".format(rf_random_f1_score*100))

Random Forest Hyperparameters Tuning Using Randomized Search
Precision: 72.857%
Accuracy:  78.771%
Recall:    72.857%
F1-Score:  72.857%


In [52]:
tn, fp, fn, tp = confusion_matrix(ytest, pred_rf_rand).ravel()
conf_matrix=pd.DataFrame({"pred_Survived":[tp,fp],"pred_Not Survived":[fn,tn]},index=["Survived","Not Survived"])
conf_matrix

,pred_Survived,pred_Not Survived
Survived,51,19
Not Survived,19,90


## 3. Submission on Kaggle

### Task 4: Import test data

In [53]:
#write code here
test = pd.read_csv('competition_clean.csv')
df_test = test.copy()
df_test.head()


,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Title,GrpSize,FareCat,AgeCat
0,892,3,male,0,0,Q,Mr,solo,0-10,32-48
1,893,3,female,1,0,S,Mrs,couple,0-10,32-48
2,894,2,male,0,0,Q,Mr,solo,0-10,48-64
3,895,3,male,0,0,S,Mr,solo,0-10,16-32
4,896,3,female,1,1,S,Mrs,group,10-25,16-32


### Task 5: Do One Hot encoding of test data

In [54]:
#write code here
df_OneHot=pd.get_dummies(df_test,columns=['Pclass','Sex','Embarked','Title','GrpSize','FareCat','AgeCat'])
df_OneHot.head()

,PassengerId,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,FareCat_10-25,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
0,892,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
1,893,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,894,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,895,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,896,1,1,0,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


### Task 6: Separate Passenger ID for submission

In [55]:
#Write code here
PassengerID = df_OneHot['PassengerId']
xinput = df_OneHot.drop(columns = 'PassengerId', axis=1)
xinput

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,FareCat_10-25,FareCat_100+,FareCat_25-40,FareCat_40-70,FareCat_70-100,AgeCat_0-16,AgeCat_16-32,AgeCat_32-48,AgeCat_48-64,AgeCat_64+
0,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,1,1,0,0,1,1,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
414,0,0,1,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
415,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
416,0,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


### Task 7: Do prediction through final model

In [56]:
#write code here
pred_final= clf_rf_rand.predict(xinput)
pred_final.shape

(418,)

#### Creating Data Frame for submission

In [57]:
submission=pd.DataFrame({'PassengerId':PassengerID,'Survived':pred_final})

In [58]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [59]:
submission.set_index(submission.columns[0],inplace= True)

### Task 8: Export the dataset into csv file

In [60]:
# Write code here
submission.to_csv('Submission_EasyEnsemble_ver_2.csv')